In [1]:
import pandas as pd
import csv 
import keras
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard, Callback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
PER_range = 878
PER_range_raw = PER_range

API_range = 1879
API_range_raw = API_range

loopcount = 1

In [4]:
train_df = pd.read_csv(r"TaiLieu/train-0.csv")
val_df   = pd.read_csv(r"TaiLieu/file-0.csv")#, header=None, skiprows=1).sample(frac=1)
test_df  = pd.read_csv(r"TaiLieu/file-1.csv")#, header=None, skiprows=1).sample(frac=1)

print(train_df)

           0    1    2    3    4    5    6    7    8    9  ...  1869  1870  \
0        0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1        0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  ...   1.0   1.0   
2        0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3        0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...   1.0   1.0   
4        0.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  ...   1.0   1.0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
21405  225.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  ...   1.0   0.0   
21406  225.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  ...   1.0   0.0   
21407  225.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0  ...   0.0   1.0   
21408  226.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0  ...   0.0   0.0   
21409  226.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   0.0   0.0   

       1871  1872  1873  1874  1875  1876  1877  1878  
0      

In [5]:
class Metrics(Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y if (y.ndim == 1 or y.shape[1] == 1) else np.argmax(y, axis=1)
        self.reports = []

    def on_epoch_end(self, epoch, logs={}):
        y_hat = np.asarray(self.model.predict(self.x))
        y_hat = np.where(y_hat > 0.5, 1, 0) if (y_hat.ndim == 1 or y_hat.shape[1] == 1)  else np.argmax(y_hat, axis=1)
        report = classification_report(self.y,y_hat,output_dict=True)
        self.reports.append(report)
        return
   
    # Utility method
    def get(self, metrics, of_class):
        return [report[str(of_class)][metrics] for report in self.reports]



In [6]:
def readData(JMP_STEP):

	global PER_range, API_range, loopcount

	PER_range_use = PER_range
	API_range_use = API_range

	PER_range_use -= JMP_STEP * loopcount
	API_range_use -= JMP_STEP * loopcount

	loopcount += 1
	try:
		train_df_2 = train_df.drop([str(i) for i in range(PER_range_use, PER_range)], axis = 1)
		train_df_2 = train_df_2.drop([str(i) for i in range(API_range_use, API_range)], axis = 1)
		#print((train_df_2))
		val_df_2 = val_df.drop([str(i) for i in range(PER_range_use, PER_range)], axis = 1)
		val_df_2 = val_df_2.drop([str(i) for i in range(API_range_use, API_range)], axis = 1)

		test_df_2 = test_df.drop([str(i) for i in range(PER_range_use, PER_range)], axis = 1)
		test_df_2 = test_df_2.drop([str(i) for i in range(API_range_use, API_range)], axis = 1)
		#print(train_df_2)

		train_y = np.array(train_df_2.iloc[:, 0]) #lấy cột 0 là cột nhãn 
		train_x = np.array(train_df_2.iloc[:,1:]) #lấy cột 1 làm đặc trưng
		unique, counts = np.unique(train_y, return_counts=True)
		#print("LABEL TRAIN: ", len(unique))

		val_y = np.array(val_df_2.iloc[:, 0]) #lấy cột 0 là cột nhãn 
		val_x = np.array(val_df_2.iloc[:,1:]) #lấy cột 1 làm đặc trưng
		unique_v, counts_ = np.unique(val_y, return_counts=True)
		#print("LABEL VAL: ", len(unique_v))

		test_y = np.array(test_df_2.iloc[:, 0]) #lấy cột 0 là cột nhãn 
		test_x = np.array(test_df_2.iloc[:,1:]) #lấy cột 1 làm đặc trưng
		unique_t, counts_t = np.unique(test_y, return_counts=True)
		#print("LABEL TEST: ", len(unique_t))

		SIZE = (len(train_df_2.columns) - 1)

		BATCH_SIZE = 32
	#SIZE = API_range_use - 1 - JMP_STEP*2
		SIZE2= 1
		N_CLASSES = 228
		LR = 0.001
		N_EPOCHS = 1

		#print(train_x.shape,train_y.shape)
		#print(val_x.shape,val_y.shape)
		#print(test_x.shape,test_y.shape)

		train_x = train_x.reshape(train_x.shape[0], SIZE2, SIZE, 1)

		val_x = val_x.reshape(val_x.shape[0], SIZE2, SIZE, 1)
		test_x = test_x.reshape(test_x.shape[0], SIZE2, SIZE, 1)

		LABELS = np.unique(train_y)
		original_test_y = test_y

		train_y = to_categorical(train_y, N_CLASSES)
		val_y = to_categorical(val_y, N_CLASSES)
		test_y = to_categorical(test_y, N_CLASSES)

		#print(LABELS)



		input0 = Input(shape=(SIZE2,SIZE,1))
		conv1 = Conv2D(32, kernel_size=2, activation='relu', padding="same" ,input_shape=(SIZE2, SIZE, 1))(input0)
		pool1 = MaxPooling2D((1, 2), padding = 'same')(conv1)
		conv2 = Conv2D(32, kernel_size=2, activation='relu', padding="same")(pool1)
		pool2 = MaxPooling2D((1, 2), padding = 'same')(conv2)

		conv3 = Conv2D(64, kernel_size=2, activation='relu', padding="same")(pool2)
		pool3 = MaxPooling2D((1, 2), padding = 'same')(conv3)
		'''
		#conv4 = Conv2D(64, kernel_size=2, activation='relu', padding="same")(pool3)
		#pool4 = MaxPooling2D((2, 2), padding = 'same')(conv4)
		'''
		flatten_per = Flatten()(pool3)

		hidden1 = Dense(1024, activation='relu')(flatten_per)
		output = Dense(N_CLASSES, activation='softmax')(hidden1)

		model = Model(inputs=input0, outputs=output)
		model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

		metrics_multiclass = Metrics(train_x, train_y)
		history = model.fit(train_x, train_y, epochs=N_EPOCHS, batch_size = BATCH_SIZE ,validation_data=(val_x, val_y), callbacks=[metrics_multiclass])
	except Exception as exc:
		raise exc


In [8]:
i = 1
JMP_STEP = 200


In [9]:
while(PER_range_raw > 100):
	if(PER_range_raw > 100):
		print("lan loop thu ", i)
		i += 1
		print("PER_range = ", PER_range_raw)
		print("API_range = ", API_range_raw)
		first = time.time()
		readData(JMP_STEP)
		second = time.time()
		print("Thoi gian chay: ", round(second - first, 2), "s")
	
	PER_range_raw -= JMP_STEP
	API_range_raw -= JMP_STEP



lan loop thu  1
PER_range =  878
API_range =  1879
670/670 [==============================] - 217s 320ms/step - loss: 0.4544 - accuracy: 0.8956 - val_loss: 0.1354 - val_accuracy: 0.9661


C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Thoi gian chay:  219.39 s
lan loop thu  2
PER_range =  678
API_range =  1679
670/670 [==============================] - 138s 205ms/step - loss: 0.4833 - accuracy: 0.8854 - val_loss: 0.1375 - val_accuracy: 0.9653


C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Thoi gian chay:  139.12 s
lan loop thu  3
PER_range =  478
API_range =  1479
670/670 [==============================] - 83s 124ms/step - loss: 0.5259 - accuracy: 0.8765 - val_loss: 0.1310 - val_accuracy: 0.9702


C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Thoi gian chay:  83.75 s
lan loop thu  4
PER_range =  278
API_range =  1279
670/670 [==============================] - 28s 41ms/step - loss: 0.6815 - accuracy: 0.8293 - val_loss: 0.2182 - val_accuracy: 0.9336
Thoi gian chay:  27.94 s


C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [10]:
print("Done")

Done
